In [17]:
import tensorflow as tf
import pickle
import numpy as np

In [22]:
xtrain=pickle.load(open("X_train.pickle","rb"))
ytrain=pickle.load(open("Y_train.pickle","rb"))
xtest=pickle.load(open("X_test.pickle","rb"))
ytest=pickle.load(open("Y_test.pickle","rb"))

In [23]:
xtest.shape,ytest.shape

((3000, 100, 100, 3), (3000,))

In [37]:
def img_vectorize(imgData):
    imgvector=imgData.reshape(imgData.shape[0],-1).T
    return imgvector

In [38]:
#Flattening and reshaping data
xtrain_flat=img_vectorize(xtrain)
xtest_flat=img_vectorize(xtest)
ytrain_flat=np.reshape(ytrain,(1,-1))
ytest_flat=np.reshape(ytest,(1,-1))

#Normalize x data
xtrain_flat=xtrain_flat/255
xtest_flat=xtest_flat/255

In [44]:
def one_hot_encode(y,classes):
    c=tf.constant(classes,name="classes")
    one_hot_matrix=tf.one_hot(y,c,axis=0)
    sess=tf.Session()
    one_hot=sess.run(one_hot_matrix)
    sess.close()
    return one_hot

In [45]:
#One hot encoding y
ytrain_flatHot=one_hot_encode(ytrain,6)
ytest_flatHot=one_hot_encode(ytest,6)

array([2, 3, 1, ..., 5, 3, 0])

In [46]:
ytrain_flatHot

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [49]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    m = X.shape[1] 
    mini_batches = []
    np.random.seed(seed)
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((Y.shape[0],m))

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

In [50]:
def create_placeholders(n_x,n_y):
    X=tf.placeholder(tf.float32,[n_x,None])
    Y=tf.placeholder(tf.float32,[n_y,None])
    return X,Y

In [102]:
def initialize_params(dims):
    parameters={}
    for i in range(1,len(dims)):
        wstr='W'+str(i)
        bstr='b'+str(i)
        parameters[wstr]=tf.get_variable([dims[i],dims[i-1]],initializer = tf.contrib.layers.xavier_initializer(seed = 1))
        parameters[bstr]=tf.get_variable([dims[i],1],initializer =  tf.zeros_initializer())
    return parameters

In [103]:
def forward_propagation(X,parameters,acts):
    
    L = len(parameters) // 2
    A=X
    
    for i in range(1,L):
        A_prev=A
        Z=tf.add(tf.matmul(parameters['W'+str(i)],A_prev),parameters['b'+str(i)])
        act=acts[i-1]
        if act=='sigmoid':
            A=tf.math.sigmoid(Z)
        if act=='ReLu':
            A=tf.nn.relu(Z)
        if act=='tanh':
            A=tf.math.tanh(Z)
        if act=='leakyReLu':
            A=tf.nn.leaky_relu(Z, alpha=0.2)
    ZL=tf.add(tf.matmul(parameters['W'+str(L)],A_prev),parameters['b'+str(L)])
    return ZL

In [104]:
def cost_compute(ZL):
    logits = tf.transpose(ZL)
    labels = tf.transpose(Y)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = labels))
    return cost

In [105]:
def model(X_train, Y_train, X_test, Y_test,learning_rate = 0.0001,
          num_epochs = 1500, minibatch_size = 32, print_cost = True):
    
    n=input("Enter number of layers including softmax : ")
    dims = [int(item) for item in input("Enter the number of units in each layer : ").split()] 
    dims.insert(0,xtrain.shape[0])
    print("sigmoid tanh ReLu leakyReLu")
    acts= [lol for lol in input("Enter the activation functions for each layer : ").split()]
    costs=[]
    m=xtrain.shape[1]
    n_x=xtrain.shape[0]
    n_y=ytrain.shape[0]
    #Creating Placeholders
    X,Y=create_placeholders(n_x,n_y)
    
    #Initialising parameters
    parameters = initialize_params(dims)
    
    #Forward Propagation
    ZL=forward_propagation(X_train,parameters,acts)
    
    #Compute Cost
    cost=cost_compute(ZL,Y)
    
    #Optimization
    optimizer = tf.train.AdamOptimizer(learning_rate =learning_rate).minimize(cost)
    
    ##Running tensorflow session
    #Initialize all the variables
    init = tf.global_variables_initializer()
    
    with tf.Session as sess:
        sess.run(init)
        
        for epoch in range(epoch):
            epoch_cost = 0.                      
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            seed = seed + 1
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)

            for minibatch in minibatches:
                # Select a minibatch
                (minibatch_X, minibatch_Y) = minibatch
                _ , minibatch_cost = sess.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})
                epoch_cost += minibatch_cost / minibatch_size

            # Print the cost every epoch
            if print_cost == True and epoch % 100 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            if print_cost == True and epoch % 5 == 0:
                costs.append(epoch_cost)
                
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per fives)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()

    #Saving the parameters in a variable
    parameters = sess.run(parameters)

    # Calculate the correct predictions
    correct_prediction = tf.equal(tf.argmax(ZL), tf.argmax(Y))

    # Calculate accuracy on the test set
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

    print ("Train Accuracy:", accuracy.eval({X: X_train, Y: Y_train}))
    print ("Test Accuracy:", accuracy.eval({X: X_test, Y: Y_test}))

    return parameters

In [106]:
parameters = model(xtrain_flat, ytrain_flatHot, xtest_flat, ytest_flatHot)

Enter number of layers including softmax : 4
Enter the number of units in each layer : 25 12 6
sigmoid tanh ReLu leakyReLu
Enter the activation functions for each layer : ReLu ReLu ReLu


TypeError: unhashable type: 'list'